In [231]:
import cv2, glob, pickle, csv
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [232]:
image_folder_path = "/home/openroot/Tanmoy/Working Stuffs/myStuffs/ISIC-2017/ISIC-2017/"
image_extension = ".jpg"

num_image_files = 0
pixel_depth = 255.0
num_labels = 2
rows, cols = 70, 100
img_channels = 3

resized_folder_path = "/home/openroot/Tanmoy/Working Stuffs/myStuffs/havss-tf/ISIC-2017/resized/"
resized_image_size = (100, 70)

In [233]:
def resizeImage():
    image_files = glob.glob(image_folder_path + "*" + image_extension)
    for image in image_files:
        try:
            image_name = image.split("/")[-1]
            image = cv2.imread(image, cv2.IMREAD_UNCHANGED)
            image = cv2.resize(image, resized_image_size)
            cv2.imwrite(resized_folder_path +  image_name, image)
        except Exception as e:
            print("Unable To Reize {0}".format(image_name))

In [234]:
def imagePickle(images):
    try:
        pickle_file = resized_folder_path + "imageDataset.pickle"
        with open(pickle_file, "wb") as f:
            pickle.dump(images, f, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print("Error While Creating Pickle File {0}".format(e))

In [235]:
def loadPickle():
    try:
        pickle_file = resized_folder_path + "imageDataset.pickle"
        with open(pickle_file, "rb") as f:
            dataset = pickle.load(f)
    except Exception as e:
        print("Error While Loading Pickle File {0}".format(e))
    return dataset

In [248]:
def readCSV():
    label_dataset = []
    label_names = []
    label_file = resized_folder_path + "ISIC-2017-label.csv"
    count_non_cancer = 0
    try:
        with open(label_file, "rb") as f:
            reader = csv.DictReader(f)
            for row in reader:
                if not int(row["melanoma"]):
                    count_non_cancer += 1
                    if count_non_cancer > 480:
                        continue
                label_names = np.append(label_names, row["image_id"])
                label_dataset = np.append(label_dataset, [float(row["melanoma"])])
    except Exception as e:
        print("Error While Reading CSV Label File: {0}".format(e))
    return label_dataset, label_names

In [250]:
def formatCSV(data):
    labels = (np.arange(num_labels) == data[:,None]).astype(np.float32)
    return labels

In [251]:
def createDataset(image_dataset, labels_dataset):
    total_images = len(image_dataset)
    print("Total No. Of Images {0}".format(total_images))
    no_test = 200
    print("Division Of Dataset {0}".format(no_test))
    test_images = image_dataset[0:no_test, :, :, :]
    test_labels = labels_dataset[0:no_test, :]
    train_images = image_dataset[no_test:, :, :, :]
    train_labels = labels_dataset[no_test:, :]
    return train_images, train_labels, test_images, test_labels

In [252]:
def main():
    print("Resizing Images")
#     resizeImage()
    
    print("Reading CSV Label File")
    csv_data, csv_data_names = readCSV()
    
    print("Formating Labels")
    labels_x = formatCSV(csv_data)
    print csv_data.shape
    print labels_x.shape
    print labels_x[0:10]
    
    print("Creating Image Array")
#     image_x = np.array([cv2.imread(resized_folder_path + image + image_extension) for image in csv_data_names])    
    image_x = np.array([(cv2.imread(resized_folder_path + image + image_extension).astype(float) - pixel_depth / 2) / pixel_depth for image in csv_data_names])
    print image_x.shape
    
    print("Saving Image Array In Pickle Form - imageDataset.pickle")
    imagePickle(image_x)
    
#     print image_x[0][2][0]
#     image_x[0][2][0][0] = 0
#     print image_x[0][2][0]
    
#     Check Equality
#     dataset = loadPickle()
#     print np.array_equal(image_x, dataset)

    print("Creating Training And Testing Dataset")
    train_images, train_labels, test_images, test_labels = createDataset(image_x, labels_x)
    
    
    print("Length Of Training Image Data {0} Shape {1}".format(len(train_images), train_images.shape))
    print("Length Of Training Label Data {0} Shape {1}".format(len(train_labels), train_labels.shape))
    print("Length Of Test Image Data {0} Shape {1}".format(len(test_images), test_images.shape))
    print("Length Of Test Label Data {0} Shape {1}".format(len(test_labels), test_labels.shape))
    
    print("Saving All Dataset In Pickle Form - ISIC.pickle")
    pickle_file = resized_folder_path + "ISIC.pickle"
    try:
        with open(pickle_file, "wb") as f:
            save = {
                'train_dataset': train_images,
                'train_labels': train_labels,
                'test_dataset': test_images,
                'test_labels': test_labels,
            }
            pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print("Error While Saving ISIC Pickle {0}".format(e))
        
    print labels_x[0:10, :]
    print test_labels[0:10, :]
    
    print("Mini Dataset")
    print train_labels[0: 10, :]
    
    
#     img = Image.fromarray(test_images[0], 'RGB')
#     img.save('/home/openroot/Tanmoy/Working Stuffs/myStuffs/havss-tf/my.png')
#     img.show()

In [253]:
main()

Resizing Images
Reading CSV Label File
Formating Labels
(848,)
(848, 2)
[[ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
Creating Image Array
(848, 70, 100, 3)
Saving Image Array In Pickle Form - imageDataset.pickle
Creating Training And Testing Dataset
Total No. Of Images 848
Division Of Dataset 200
Length Of Training Image Data 648 Shape (648, 70, 100, 3)
Length Of Training Label Data 648 Shape (648, 2)
Length Of Test Image Data 200 Shape (200, 70, 100, 3)
Length Of Test Label Data 200 Shape (200, 2)
Saving All Dataset In Pickle Form - ISIC.pickle
[[ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
[[ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
Mini Dataset
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
